# Pipeline 

In [1]:
%load_ext autoreload
%autoreload 2

from averitec import Datapoint
from evidence_generation import GptEvidenceGenerator, GptBatchedEvidenceGenerator, DynamicFewShotBatchedEvidenceGenerator
from classification import DefaultClassifier, HuggingfaceClassifier, AverageEnsembleClassifier, LogRegEnsembleClassifier
from retrieval import SimpleFaissRetriever, Retriever, MmrFaissRetriever, SubqueryRetriever
from pipeline import Pipeline, MockPipeline
import pickle
from labels import label2id, id2label
import numpy as np
from sklearn.metrics import classification_report
import random
from tqdm import tqdm
import htmldate
random.seed(111)

import json

## Pipeline test

In [2]:
split = "val"
path = "/mnt/data/factcheck/averimatec/"
with open(path + f"{split}.json") as f:
    dataset = json.load(f)
    for i in range(len(dataset)):
        dataset[i]["claim_id"] = i
        dataset[i]["split"] = split
    datapoints = [Datapoint.from_dict(d) for d in dataset]

In [3]:
datapoint = Datapoint.from_dict(dataset[150])
datapoint

Datapoint(claim='Images show Tanks & Troops  deploying inside USA & Canada', claim_id=150, claim_date='2023-06-16', claim_images=['67d0a15035531374f6dae467#CLAIM#0.jpg', '67d0a15035531374f6dae467#CLAIM#1.jpg', '67d0a15035531374f6dae467#CLAIM#2.jpg'], speaker='Hal Turner', original_claim_url='https://web.archive.org/web/20230617224616/https://halturnerradioshow.com/index.php/en/news-page/world/tanks-troops-deploying-inside-u-s-a-and-canada', reporting_source='halturnerradioshow.com', location_ISO_code='US', label='Refuted', metadata={'transcription': '', 'media_source': 'https://web.archive.org/web/20230618011405im_/https://halturnerradioshow.com/images/2023/06/17/Tank-Train-Montreal-06-17-2023_large.jpg,https://web.archive.org/web/20230618011405im_/https://halturnerradioshow.com/images/2023/06/17/MilitaryConvoyOnFreeway-in-LA-CA.jpg,,https://web.archive.org/web/20230618011405im_/https://halturnerradioshow.com/images/2023/06/17/MilitaryConvoyOnFreeway-in-LA-CA.jpg', 'claim_types': ['Med

In [4]:
print(datapoint.speaker)

Hal Turner


In [5]:
# retriever = SimpleFaissRetriever(path="/mnt/data/factcheck/averitec-data/data_store/vecstore/dev/6k")
retriever = MmrFaissRetriever(path=f"/mnt/data/factcheck/averimatec/vector_store/{split}/text/2k_mxbai",k=7, fetch_k=24)
# retriever = MmrFaissRetriever(path=f"/mnt/data/factcheck/averitec-data/data_store/vecstore/{split}/2k")
retrieval_result = retriever(datapoint)
retrieval_result

RetrievalResult(documents=[Document(id='183e25ce-02d3-4b08-8511-f1da10c972ff', metadata={'url': 'https://www.army.mil/article/169006/us_uk_forces_join_canada_armys_largest_military_exercise', 'context_before': 'In the grand scheme, Maple Resolve strengthens the bonds between these three allies and provides a glimpse into the future of military conflict. Wars of the future will continue to involve the collaboration of multiple nations - no one fights alone anymore. A key element in this joint exercise is the exchange of aviation assets. This particular night has Canadian piloted CH-47 Chinooks and U.S. manned UH-60 Black Hawk helicopters. They will be working together to transport Canadian infantryman from CFB Cold Lake to attack nearby objectives. At these sites the Canadians will fight the opposing force - played by the British military. Outside a massive jet hangar stands an equally large man named Maj. Cole Petersen. An infantryman hailing from Cornell, British Columbia, he is the D

In [10]:
retrieval_result.images

[[{'url': 'https://abcnews.go.com/US/state-suspends-brooklyn-funeral-homes-licence-stored-dozens/story?id=70451543',
   'imageUrl': 'https://s.abcnews.com/images/US/200430_wabc_funeralhomes_hpMain_4x5_992.jpg',
   'title': "State suspends Brooklyn funeral home's license after it stored dozens of bodies in rental trucks - ABC News",
   'thumbnailUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQBwlh2cc2D9Nni1Te0y1QqCglJLpveaCNk_yz1KWPSaw5cnULX',
   'content': '[breakingAppeals court disqualifies Alina Habba from serving as US attorney for New Jersey](https://abcnews.go.com/US/appeals-court-disqualifies-alina-habba-serving-us-attorney/story?id=127999613)\n\n# State suspends Brooklyn funeral home\'s license after it stored dozens of bodies in rental trucks\n\nThe state health department and Brooklyn DA seek answers.\n\nBy[Aaron Katersky](https://abcnews.go.com/author/aaron_katersky), [Mark Crudele](https://abcnews.go.com/author/mark_crudele), and [Ivan Pereira](https://abcnews

In [8]:
from classification import DefaultClassifier, HuggingfaceClassifier, AverageEnsembleClassifier, LogRegEnsembleClassifier, RandomForestClassifier, NoTiebreakClassifier

# target = path + "data_store/vecstore/test/2k"
PIPELINE_NAME = "mmr+gpt51-dfewshot-atype"
classifier = NoTiebreakClassifier()  # DefaultClassifier()
if False:
    pipeline = MockPipeline(
        dumps=f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_mmr+gpt4o-dfewshot-tiebrk-atype.pkl",
        classifier=NoTiebreakClassifier()
    )
else:
    pipeline = Pipeline(
        #dumps = "/mnt/data/factcheck/averitec-data/data_store/submissions/dev_mmr+gpt4o-dfewshot.pkl",
        #SubqueryRetriever(retriever),
        retriever,
        evidence_generator=DynamicFewShotBatchedEvidenceGenerator(), 
        classifier=classifier
    )

In [9]:
list(enumerate(retrieval_result.images))

[(0,
  [{'url': 'https://www.wranglerforum.com/threads/trickle-charger-mounting-position.2449945/page-2',
    'imageUrl': 'https://www.wranglerforum.com/attachments/20220612_122537-jpg.4564419/?auto=webp&fit=bounds&format=pjgp&height=1920&optimize=high&width=1920',
    'title': 'Trickle charger mounting position | Page 2 | Jeep Wrangler Forum',
    'thumbnailUrl': 'https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQsQvwcvv_wJRUDnT60Nr4dgPP4dVd0ahlJeroQGDQmAuYDESUB',
    'content': '[![Jeep Wrangler Forum banner](https://images.platforum.cloud/banners/wranglerforum_com_banner_2880w.jpg?auto=webp&fit=bounds&format=pjgp&optimize=high&quality=50)](https://www.wranglerforum.com/)\n\nMenu\n\n\n[Log in](https://www.wranglerforum.com/login/)\n\n* * *\n\n[Sign up](https://www.wranglerforum.com/register/)\n\n* * *\n\n- [Home](https://www.wranglerforum.com/)\n- [Forums](https://www.wranglerforum.com/forums/)\n- [Model Specific Jeep Discussions](https://www.wranglerforum.com/forums/#model-spec

In [11]:
PIPELINE_NAME = "evtext"

In [12]:
split

'val'

In [13]:
submission = []
dump = []

for dp in tqdm(datapoints):
    pipeline_result = pipeline(dp)
    submission.append(pipeline_result.to_submission())
    dump.append(pipeline_result)
with open(f"{path}submissions/{split}_{PIPELINE_NAME}.json", "w") as f:
    json.dump(submission, f, indent=4)
with open(f"{path}submissions/{split}_{PIPELINE_NAME}.pkl", "wb") as f:
    pickle.dump(dump, f)

100%|██████████| 152/152 [00:05<00:00, 26.48it/s]


In [14]:
files = pipeline.evidence_generator.get_batch_files(path=f"/mnt/data/factcheck/averimatec/batch_jobs/{split}_{PIPELINE_NAME}", batch_size=352)

In [15]:
files

['/mnt/data/factcheck/averimatec/batch_jobs/val_evtext/batch_1.jsonl']

In [ ]:
batch_results = pipeline.evidence_generator.submit_and_await_batches(
    files, f"/mnt/data/factcheck/averimatec/batch_jobs/{split}_{PIPELINE_NAME}/output.jsonl"
)

  0%|          | 0/1 [00:00<?, ?it/s]

Batch(id='batch_69309e5776988190a40b3d53b28847c0', completion_window='24h', created_at=1764793943, endpoint='/v1/chat/completions', input_file_id='file-1PquAjkgiaAEHHp3FNPkzn', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1764880343, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'dev-set job, batch 1'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0), model=None, usage={'input_tokens': 0, 'output_tokens': 0, 'total_tokens': 0, 'input_tokens_details': {'cached_tokens': 0}, 'output_tokens_details': {'reasoning_tokens': 0}})
waiting for batch to complete BatchRequestCounts(completed=0, failed=0, total=0) batch_69309e5776988190a40b3d53b28847c0
waiting for batch to complete BatchRequestCounts(completed=0, failed=0, total=0) batch_69309e5776988190a40b3d53b28847c0
waiting for batch to complete BatchRequestCoun

In [35]:
with open(f"/mnt/data/factcheck/averimatec/batch_jobs/{split}_{PIPELINE_NAME}/output.jsonl") as f:
    batch_results = [json.loads(line)["response"]["body"]["choices"][0]["message"]["content"] for line in f]

In [ ]:
new_result = pipeline.evidence_generator.update_pipeline_result(pipeline_result, batch_results, pipeline.classifier)

In [36]:
new_dump = []
pipeline.evidence_generator.fallback_gpt_generator.client.temperature = 0.5
for pipeline_result, batch_result in zip(dump[: len(batch_results)], batch_results):

    new_result = pipeline.evidence_generator.update_pipeline_result(
        pipeline_result, batch_result, pipeline.classifier
    )
    new_dump.append(new_result)

FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL


In [ ]:
new_result.to_submission()

In [37]:
new_dump = []
pipeline.evidence_generator.fallback_gpt_generator.client.temperature = .5
for pipeline_result, batch_result in zip(dump[:len(batch_results)], batch_results):
    new_result = pipeline.evidence_generator.update_pipeline_result(pipeline_result, batch_result, pipeline.classifier)
    new_dump.append(new_result)


In [39]:
new_result.to_submission()

{'id': 351,
 'claim': 'The national flag of India was placed at the Bangladesh University of Engineering and Technology campus gate in such a way that people could walk on it and disrespect it.',
 'questions': ['Does the viral image actually show the main gate of Bangladesh University of Engineering and Technology (BUET)?',
  'Does any source explicitly state that this image and the flag incident occurred at BUET?',
  'Does the same LinkedIn source also say that this incident was later found to be fake news by people associated with BUET?',
  'Do any of the provided sources independently confirm (beyond the LinkedIn caption) that students at BUET drew or placed an Indian national flag on the ground to be stepped on?',
  'Do any of the provided sources report on this image as part of a verified news story about disrespecting the Indian flag at BUET?',
  'According to the LinkedIn source, what is the status of the claim that students at BUET stepped on an Indian flag drawn on the ground?

In [43]:
with open(f"{path}submissions/{split}_{PIPELINE_NAME}2.json", "w") as f:
    json.dump([d.to_submission() for d in new_dump], f, indent=4)
with open(f"{path}submissions/{split}_{PIPELINE_NAME}2.pkl", "wb") as f:
    pickle.dump(new_dump, f)

In [41]:
f"{path}submissions/{split}_{PIPELINE_NAME}.json"

'/mnt/data/factcheck/averimatec/submissions/test_base64.json'

In [ ]:
(
    new_dump[1],
    new_dump[1].evidence_generation_result,
    new_dump[1].classification_result
)

In [ ]:
with open(f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.json", "w") as f:
    json.dump([d.to_submission() for d in new_dump], f, indent=4)
with open(f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.pkl", "wb") as f:
    pickle.dump(new_dump, f)

In [ ]:
with open(f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.json", "w") as f:
    json.dump([d.to_submission() for d in new_dump], f, indent=4)
with open(f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.pkl", "wb") as f:
    pickle.dump(new_dump, f)

## collapsible begin

In [ ]:
from IPython.display import display, Markdown, Latex

In [ ]:
knn_retrieval_result = retriever(datapoint)
display(Markdown("### 🗯️ " + datapoint.claim))
display(Markdown("*Retrieved by knn*\n\n"))
# sample 3
for r in knn_retrieval_result:
    newline = "\n"
    display(Markdown(f"**{r.metadata['url']}**\n\n{r.page_content[:256]}"))

In [ ]:
from retrieval import MmrFaissRetriever

mmr_retriever = MmrFaissRetriever(retriever.path)
mmr_retrieval_result = mmr_retriever(datapoint)
display(Markdown("### 🗯️ " + datapoint.claim))
display(Markdown("*Retrieved by MMR*\n\n"))
# sample 3
for r in mmr_retrieval_result:
    newline = "\n"
    display(Markdown(f"**{r.metadata['url']}**\n\n{r.page_content[:256]}"))

In [ ]:
subquery_retriever = SubqueryRetriever(retriever)
subquery_retrieval_result = subquery_retriever(datapoint)
display(Markdown("### 🗯️ " + datapoint.claim))
display(Markdown("*Retrieved by subqueries*\n\n"))
# sample 3
for r in subquery_retrieval_result:
    newline = "\n"
    display(Markdown(f"**{r.metadata['url']}**\n\n*{';'.join(r.metadata['queries'])}*\n\n{r.page_content[:256]}"))

In [ ]:
subquery_retrieval_result.metadata

## Collapsible section end

In [ ]:
evidence_generator = GptBatchedEvidenceGenerator("gpt-4o")
evidence_generation_result = evidence_generator(datapoint, retrieval_result)
evidence_generation_result

In [ ]:
evidence_generation_result.metadata["suggested_label"]

In [ ]:
datapoint.label

In [ ]:
classifier = DefaultClassifier()
classification_result = classifier(datapoint, evidence_generation_result, retrieval_result)
str(classification_result), classification_result

In [ ]:
datapoint2 = Datapoint.from_dict(dataset[16])
pipeline = Pipeline(retriever, evidence_generator, classifier)
pipeline_result = pipeline(datapoint2)
pipeline_result

In [ ]:
str(pipeline_result.classification_result), datapoint2.label

In [ ]:
pipeline_result.to_submission()

In [ ]:
# pickle dump pipeline result
import pickle
with open('data/pipeline_result.pkl', 'wb') as f:
    pickle.dump(pipeline_result, f)

In [ ]:
%run src/prediction/evaluate_veracity.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submission_dev_avg_clf.json

In [ ]:
import json, os
# crawl /mnt/data/factcheck/averitec-data/data_store/batch_jobs and each time you find gpt4o in folder name and "output" in filename, load the file and add its line count to line_counts
line_counts = []
for root, dirs, files in os.walk("/mnt/data/factcheck/averitec-data/data_store/batch_jobs"):
    #print(root, files)
    if "gpt4o" in root:
        for f in files:
            if "output" in f:
                print(os.path.join(root,f))
                with open(os.path.join(root,f)) as f:
                    line_counts.append(len(f.readlines()))
            

In [ ]:
line_counts

In [ ]:
()/sum(line_counts)